In [2]:
%load_ext sql
%sql sqlite:///chinook.db

## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (судить по количеству купленных треков).

In [2]:
%%sql
SELECT
    c.Country AS MostPopularCountry
FROM
    customers c
JOIN
    invoices i ON c.CustomerId = i.CustomerId
JOIN
    invoice_items ii ON i.InvoiceId = ii.InvoiceId
JOIN
    tracks t ON ii.TrackId = t.TrackId
JOIN
    artists a ON t.AlbumId = a.ArtistId
WHERE
    a.Name = 'Iron Maiden'
GROUP BY
    c.Country
ORDER BY
    COUNT(ii.quantity) DESC
LIMIT 1;

 * sqlite:///chinook.db
Done.


MostPopularCountry
USA


## Задание 2
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом (цена альбома - сумма цен треков).

In [19]:
%%sql
SELECT
    a.Title AS AlbumTitle,
    ROUND(SUM(t.UnitPrice), 2) AS AlbumPrice,
    SUM(t.Milliseconds) / 60000.0 AS AlbumDurationMinutes
FROM
    albums a
JOIN
    tracks t ON a.AlbumId = t.AlbumId
GROUP BY
    a.AlbumId, a.Title
HAVING
    AlbumPrice > (SELECT AVG(AlbumPrice) FROM (
        SELECT
            a1.AlbumId,
            SUM(t1.UnitPrice) AS AlbumPrice
        FROM
            albums a1
        JOIN
            tracks t1 ON a1.AlbumId = t1.AlbumId
        GROUP BY
            a1.AlbumId
    ) AS AvgAlbumPrice)
ORDER BY
    AlbumDurationMinutes DESC
LIMIT 1;

 * sqlite:///chinook.db
Done.


AlbumTitle,AlbumPrice,AlbumDurationMinutes
"Lost, Season 3",51.74,1177.7597


## Задание 3
Вывести общую стоимость треков, приобретённых компаниями (только клиентами из конкретных компаний), которые обслуживались сотрудниками, нанятыми после 3 августа 2002 года.

In [20]:
%%sql
SELECT
    c.Company AS CompanyName,
    ROUND(SUM(ii.UnitPrice), 2) AS TotalPrice
FROM
    customers c
JOIN
    employees e ON c.SupportRepId = e.EmployeeId
JOIN
    invoices i ON c.CustomerId = i.CustomerId
JOIN
    invoice_items ii ON i.InvoiceId = ii.InvoiceId
WHERE
    e.HireDate > '2002-08-03' and c.Company is not null
GROUP BY
    c.Company;

 * sqlite:///chinook.db
Done.


CompanyName,TotalPrice
Banco do Brasil S.A.,37.62
Google Inc.,37.62
JetBrains s.r.o.,40.62
Microsoft Corporation,39.62
Telus,37.62
Woodstock Discos,37.62


## Задание 4
Для каждого альбома определить все жанры песен, которые в нём есть и количество песен по каждому жанру (формат вывода: альбом, жанр, количество). Выводить только те альбомы, где жанров больше одного.

In [72]:
%%sql
SELECT a.Title AS Album, g.Name AS Genre, COUNT(t.TrackId) AS Count
FROM albums a
JOIN tracks t ON a.AlbumId = t.AlbumId
JOIN genres g ON t.GenreId = g.GenreId
WHERE a.AlbumId IN (
    SELECT a.AlbumId
    FROM albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    JOIN genres g ON t.GenreId = g.GenreId
    GROUP BY a.AlbumId
    HAVING COUNT(DISTINCT g.Name) > 1
)
GROUP BY a.Title, g.Name;

 * sqlite:///chinook.db
Done.


Album,Genre,Count
"Battlestar Galactica, Season 3",Sci Fi & Fantasy,2
"Battlestar Galactica, Season 3",Science Fiction,12
"Battlestar Galactica, Season 3",TV Shows,5
Greatest Hits,Metal,14
Greatest Hits,Reggae,13
Greatest Hits,Rock,30
"Heroes, Season 1",Drama,20
"Heroes, Season 1",TV Shows,3
"LOST, Season 4",Drama,14
"LOST, Season 4",TV Shows,3


## Задание 5
Для каждого артиста (таблицы Artists), у которых есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение суммы, заработанной артистом по этому жанру к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, максимальное для артиста, то выбирается жанр раньше по алфавиту. Формат вывода: артист, жанр, соотношение.

In [21]:
%%sql
WITH genre_sales AS (
    SELECT 
        a.Name AS Artist, 
        g.Name AS Genre, 
        SUM(ii.UnitPrice * ii.Quantity) AS TotalSales
    FROM 
        Invoice_Items ii
    JOIN 
        Tracks t ON ii.TrackId = t.TrackId
    JOIN 
        Albums al ON t.AlbumId = al.AlbumId
    JOIN 
        Artists a ON al.ArtistId = a.ArtistId
    JOIN 
        Genres g ON t.GenreId = g.GenreId
    GROUP BY 
        Artist, Genre
    HAVING 
        COUNT(g.Name) > 1
),
max_genre_sales AS (
    SELECT 
        Artist, 
        MAX(TotalSales) AS MaxSales
    FROM 
        genre_sales
    GROUP BY 
        Artist
),
top_genre AS (
    SELECT 
        gs.Artist, 
        gs.Genre, 
        gs.TotalSales
    FROM 
        genre_sales gs
    JOIN 
        max_genre_sales mgs ON gs.Artist = mgs.Artist AND gs.TotalSales = mgs.MaxSales
    ORDER BY 
        gs.Artist, gs.Genre
),
top_genre_per_artist AS (
    SELECT 
        Artist, 
        MIN(Genre) AS TopGenre, 
        TotalSales
    FROM 
        top_genre
    GROUP BY 
        Artist, TotalSales
),
total_sales_per_artist AS (
    SELECT 
        Artist, 
        SUM(TotalSales) AS TotalSales
    FROM 
        genre_sales
    GROUP BY 
        Artist
)
SELECT 
    tgpa.Artist, 
    tgpa.TopGenre, 
   ROUND(tgpa.TotalSales / ts.TotalSales, 2) AS Ratio
FROM 
    top_genre_per_artist tgpa
JOIN 
    total_sales_per_artist ts ON tgpa.Artist = ts.Artist
WHERE 
    Ratio < 1
ORDER BY 
    tgpa.Artist;

 * sqlite:///chinook.db
Done.


Artist,TopGenre,Ratio
Amy Winehouse,R&B/Soul,0.67
Antônio Carlos Jobim,Latin,0.68
Audioslave,Rock,0.38
Battlestar Galactica,Science Fiction,0.5
Eric Clapton,Blues,0.68
Faith No More,Alternative & Punk,0.79
Foo Fighters,Rock,0.77
Gilberto Gil,Latin,0.65
Guns N' Roses,Rock,0.72
Heroes,Drama,0.85
